In [1]:
import os, warnings
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import gridspec
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(31415)

# set matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # cleanup cells

# load dataset
ds_train_ = image_dataset_from_directory(
    'dataset/car_or_truck/train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=1,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    'dataset/car_or_truck/valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=1,
    shuffle=False,
)

# data pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (ds_train_.map(convert_to_float).cache()
            .prefetch(buffer_size=AUTOTUNE))
ds_valid = (ds_valid_.map(convert_to_float).cache()
            .prefetch(buffer_size=AUTOTUNE))

2022-10-31 23:20:05.975744: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Found 5117 files belonging to 2 classes.


2022-10-31 23:20:07.546064: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-31 23:20:07.576243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 23:20:07.576505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2022-10-31 23:20:07.576527: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-31 23:20:07.578025: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-10-31 23:20:07.579452: I tensorflow/stream_executor/plat

Found 5051 files belonging to 2 classes.


In [2]:
import matplotlib.pyplot as plt

pretrained_base = tf.keras.models.load_model(
    'vgg16-pretrained-base',)
pretrained_base.trainable=False
'''
When doing transfer learning, it's generally not a good idea to 
retrain the entire base -- at least not without some care. 
The reason is that the random weights in the head will initially 
create large gradient updates, which propogate back into the 
base layers and destroy much of the pretraining. 
Using techniques known as fine tuning it's possible to further 
train the base on new data, but this requires some care to do well.
'''

"\nWhen doing transfer learning, it's generally not a good idea to \nretrain the entire base -- at least not without some care. \nThe reason is that the random weights in the head will initially \ncreate large gradient updates, which propogate back into the \nbase layers and destroy much of the pretraining. \nUsing techniques known as fine tuning it's possible to further \ntrain the base on new data, but this requires some care to do well.\n"

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

# attach head
model = keras.Sequential([pretrained_base,
                          layers.Flatten(),
                          layers.Dense(6, activation='relu'),
                          layers.Dense(1, activation='sigmoid'),
                         ])

In [ ]:
# training
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=10,
    verbose=0,
)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();